In [1]:
import os

os.chdir('../')

In [2]:
%pwd

'/Users/gg1313/Desktop/ML-Projects/ML_Deployment'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path


In [12]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories
from mlProject import logger


class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH, schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path
        )
        return data_transformation_config   

In [17]:
from mlProject import logger
from sklearn.model_selection import train_test_split
import pandas as pd


class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def train_test_split(self):
        data = pd.read_csv(self.config.data_path)
        train_set, test_set = train_test_split(data, test_size=0.25, random_state=42)

        train_set.to_csv(os.path.join(self.config.root_dir, "train.csv"), index=False)
        test_set.to_csv(os.path.join(self.config.root_dir, "test.csv"), index=False)

        logger.info("Train test split completed")
        logger.info(train_set.shape)
        logger.info(test_set.shape)

In [18]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_split()
except Exception as e:
    raise e

[2025-07-25 11:16:47,756: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-07-25 11:16:47,758: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-25 11:16:47,760: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-07-25 11:16:47,761: INFO: common: created directory at: artifacts]
[2025-07-25 11:16:47,761: INFO: common: created directory at: artifacts/data_transformation]
[2025-07-25 11:16:47,792: INFO: 1113866083: Train test split completed]
[2025-07-25 11:16:47,793: INFO: 1113866083: (1199, 12)]
[2025-07-25 11:16:47,794: INFO: 1113866083: (400, 12)]
